In [2]:
#Data Management

import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
from ta import add_all_ta_features

#Statistics
from statsmodels.tsa.stattools import adfuller

#Data Preprocessing

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#Supervised Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold

#Reporting
import matplotlib.pyplot as plt

# Data Ingestion

In [3]:
df = pd.read_csv('/Users/vadimsmirnov/Downloads/MLTrading/PRIMERS/SydneyHousePrices.csv')
print('Number of rows: ', df.shape[0])

Number of rows:  199504


In [4]:
df


,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house
3,2019-05-28,4,Avalon Beach,2107,1530000,3.0,1,2.0,house
4,2019-05-22,5,Whale Beach,2107,8000000,5.0,4,4.0,house
...,...,...,...,...,...,...,...,...,...
199499,2014-06-20,199500,Illawong,2234,1900000,5.0,3,7.0,house
199500,2014-05-26,199501,Illawong,2234,980000,4.0,3,2.0,house
199501,2014-04-17,199502,Alfords Point,2234,850000,4.0,2,2.0,house
199502,2013-09-07,199503,Illawong,2234,640000,3.0,2,2.0,townhouse


In [5]:
#Interpret data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199504 entries, 0 to 199503
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Date        199504 non-null  object 
 1   Id          199504 non-null  int64  
 2   suburb      199504 non-null  object 
 3   postalCode  199504 non-null  int64  
 4   sellPrice   199504 non-null  int64  
 5   bed         199350 non-null  float64
 6   bath        199504 non-null  int64  
 7   car         181353 non-null  float64
 8   propType    199504 non-null  object 
dtypes: float64(2), int64(4), object(3)
memory usage: 13.7+ MB


### Feature Engineering

In [6]:
suburb_text_unique = df['suburb'].unique()
print('Unique Suburbs: ', len(suburb_text_unique))

Unique Suburbs:  685


In [7]:
prop_type_text_unique = df['propType'].unique()
print('Unique Prop Types: ', len(prop_type_text_unique))

Unique Prop Types:  8


In [8]:
labelencoder = LabelEncoder()
encoded_suburbs = labelencoder.fit_transform(df['suburb'])
df['suburbs_encoded'] = encoded_suburbs
df.head()

,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType,suburbs_encoded
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,22
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,22
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house,654
3,2019-05-28,4,Avalon Beach,2107,1530000,3.0,1,2.0,house,22
4,2019-05-22,5,Whale Beach,2107,8000000,5.0,4,4.0,house,654


In [9]:
onehot_encoded = pd.get_dummies(df['propType'], prefix = "pt",drop_first = True)
df = df.join(onehot_encoded)
df.head(3)

,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType,suburbs_encoded,pt_duplex/semi-detached,pt_house,pt_other,pt_terrace,pt_townhouse,pt_villa,pt_warehouse
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,22,False,True,False,False,False,False,False
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,22,False,True,False,False,False,False,False
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house,654,False,True,False,False,False,False,False


## Set Target

In [10]:
#Set target
df['Target'] = df['sellPrice']
df.head(3)

,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType,suburbs_encoded,pt_duplex/semi-detached,pt_house,pt_other,pt_terrace,pt_townhouse,pt_villa,pt_warehouse,Target
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,22,False,True,False,False,False,False,False,1210000
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,22,False,True,False,False,False,False,False,2250000
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house,654,False,True,False,False,False,False,False,2920000


In [13]:
### Remove Redundant Features

df_drop = df.copy()
df_drop.drop(columns = ['Date','Id','suburb','propType','sellPrice'],inplace = True)

In [14]:
df_drop

,postalCode,bed,bath,car,suburbs_encoded,pt_duplex/semi-detached,pt_house,pt_other,pt_terrace,pt_townhouse,pt_villa,pt_warehouse,Target
0,2107,4.0,2,2.0,22,False,True,False,False,False,False,False,1210000
1,2107,4.0,3,4.0,22,False,True,False,False,False,False,False,2250000
2,2107,3.0,3,2.0,654,False,True,False,False,False,False,False,2920000
3,2107,3.0,1,2.0,22,False,True,False,False,False,False,False,1530000
4,2107,5.0,4,4.0,654,False,True,False,False,False,False,False,8000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199499,2234,5.0,3,7.0,318,False,True,False,False,False,False,False,1900000
199500,2234,4.0,3,2.0,318,False,True,False,False,False,False,False,980000
199501,2234,4.0,2,2.0,5,False,True,False,False,False,False,False,850000
199502,2234,3.0,2,2.0,318,False,False,False,False,True,False,False,640000


In [16]:
### Check for NaN or Inf Values

is_null = df_drop.isnull().values.any()
is_inf = df_drop.isin([np.inf,-np.inf]).values.any()
print("Is Null: ", is_null)
print("Is Inf: ", is_inf)

Is Null:  True
Is Inf:  False


In [20]:
#Fill Na
df_drop = df_drop.fillna(df_drop.mean())
df_drop.isna().sum()

postalCode                 0
bed                        0
bath                       0
car                        0
suburbs_encoded            0
pt_duplex/semi-detached    0
pt_house                   0
pt_other                   0
pt_terrace                 0
pt_townhouse               0
pt_villa                   0
pt_warehouse               0
Target                     0
dtype: int64

### Deature Scaling = Min Max Scaling

In [22]:
df_scaling = df_drop.copy()
Scaler = MinMaxScaler()
df_scaling.iloc[:] = Scaler.fit_transform(df_scaling)
df_scaling.head()

,postalCode,bed,bath,car,suburbs_encoded,pt_duplex/semi-detached,pt_house,pt_other,pt_terrace,pt_townhouse,pt_villa,pt_warehouse,Target
0,0.037179,0.030612,0.010204,0.025,0.032164,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000563
1,0.037179,0.030612,0.020408,0.075,0.032164,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.001048
2,0.037179,0.020408,0.020408,0.025,0.956140,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.001360
3,0.037179,0.020408,0.000000,0.025,0.032164,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000712
4,0.037179,0.040816,0.030612,0.075,0.956140,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.003725


### Train Test Split

In [25]:
#Use Correct DataFrame
is_deep_learning = False

df_tts = df.scaling.copy() if is_deep_learning else df_drop.copy()
df_tts.head()

,postalCode,bed,bath,car,suburbs_encoded,pt_duplex/semi-detached,pt_house,pt_other,pt_terrace,pt_townhouse,pt_villa,pt_warehouse,Target
0,2107,4.0,2,2.0,22,False,True,False,False,False,False,False,1210000
1,2107,4.0,3,4.0,22,False,True,False,False,False,False,False,2250000
2,2107,3.0,3,2.0,654,False,True,False,False,False,False,False,2920000
3,2107,3.0,1,2.0,22,False,True,False,False,False,False,False,1530000
4,2107,5.0,4,4.0,654,False,True,False,False,False,False,False,8000000


In [26]:
X = df_tts.iloc[:,:-1].values
y = df_tts.iloc[:,-1].values
print('X Values: \n', X[:2])
print('X Values: \n', X[:5])


X Values: 
 [[2107 4.0 2 2.0 22 False True False False False False False]
 [2107 4.0 3 4.0 22 False True False False False False False]]
X Values: 
 [[2107 4.0 2 2.0 22 False True False False False False False]
 [2107 4.0 3 4.0 22 False True False False False False False]
 [2107 3.0 3 2.0 654 False True False False False False False]
 [2107 3.0 1 2.0 22 False True False False False False False]
 [2107 5.0 4 4.0 654 False True False False False False False]]


In [28]:
#Train Test Split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state = 1, shuffle = True)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

X_train:  (179553, 12)
X_test:  (19951, 12)
y_train:  (179553,)
y_test:  (19951,)


### Machine Learning

In [29]:
# Train Regressor
regressor = RandomForestRegressor(n_estimators = 100, max_depth = 10, random_state = 0)
regressor.fit(X_train,y_train)

RandomForestRegressor(max_depth=10, random_state=0)

In [30]:
#Make predictions on Test Set
y_pred = regressor.predict(X_test)
y_pred = [round(x,0) for x in y_pred]

In [33]:
print("Test Predictions", y_pred[:5])
print("Test Actuals", y_test[:5])

Test Predictions [590029.0, 2022675.0, 1112679.0, 1045638.0, 869990.0]
Test Actuals [ 730000 1350100  860000 1390000  985000]


In [43]:
# Check Accuracy
cv = RepeatedKFold(n_splits = 5, n_repeats = 3, random_state=1)
n_scores = cross_val_score(regressor, X_train,y_train, scoring = 'neg_mean_absolute_error', cv=cv,n_jobs = -1, error_score = 'raise') 

In [45]:
#Report Performance

print('MAE Avg: ', abs(n_scores.mean()))
print('MAE Std: ', n_scores.std())

MAE Avg:  389118.30686148786
MAE Std:  20152.482105475094
